## Embeddings, Proyeccion , TSNE

In [142]:
import pandas as pd
import numpy as np
import gensim
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

In [143]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos los embeddings entrenados en la notebook anterior

In [144]:
#load model
import os
filename='/content/drive/MyDrive/Mentoria/Embedding_FinancialNews_word2vec.txt'


embeddings_index={}
f=open(os.path.join('',filename), encoding="utf-8")
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:])
  embeddings_index[word]=coefs
f.close()

In [145]:
embeddings = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=False)

In [146]:
len(embeddings.vocab)

94966

### Most Similar

In [147]:
embeddings.most_similar_cosmul('msft')

[('microsoft', 0.924543023109436),
 ('intc', 0.8724203109741211),
 ('googl', 0.8362237811088562),
 ('cmcsa', 0.8234818577766418),
 ('amzn', 0.8167346119880676),
 ('aapl', 0.811662495136261),
 ('fb', 0.8101232647895813),
 ('symc', 0.8096150159835815),
 ('sbux', 0.8077319264411926),
 ('csco', 0.8071362376213074)]

In [148]:
embeddings.most_similar('amazon' )

[('amzn', 0.7858572006225586),
 ('walmart', 0.6856096386909485),
 ('aws', 0.6330705881118774),
 ('google', 0.5992705225944519),
 ('souq', 0.5887264013290405),
 ('netflix', 0.5859094858169556),
 ('salesforce', 0.5797685384750366),
 ('jd', 0.5759837627410889),
 ('online', 0.5757834911346436),
 ('overstock', 0.5623939037322998)]

In [149]:
embeddings.similar_by_vector(embeddings.get_vector('amzn')+embeddings.get_vector('amazon'))

[('amzn', 0.951489269733429),
 ('amazon', 0.9380075931549072),
 ('walmart', 0.6383774280548096),
 ('googl', 0.6264796257019043),
 ('nflx', 0.6096340417861938),
 ('ebay', 0.5925592184066772),
 ('google', 0.5867770910263062),
 ('netflix', 0.5852630138397217),
 ('aws', 0.584559440612793),
 ('jd', 0.5729597806930542)]

## TSNE sobre tickers




Carga Dataframe News para extraer tickers

In [150]:
df_news= pd.read_csv('/content/drive/MyDrive/Mentoria/news_dataset.csv')

In [151]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221513 entries, 0 to 221512
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            221513 non-null  int64 
 1   ticker        221513 non-null  object
 2   title         221513 non-null  object
 3   category      221513 non-null  object
 4   content       221505 non-null  object
 5   release_date  221513 non-null  object
 6   provider      221513 non-null  object
 7   url           221513 non-null  object
 8   article_id    221513 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 15.2+ MB


Seleccionar ticker con mas cantidad de registros


In [152]:
df_counts = pd.DataFrame(df_news.ticker.value_counts())

In [154]:
# Seleccion de tickers con mas de 100 registros
ticker_list= list(df_counts[df_counts.ticker>100].index)

In [155]:
# Seleccion de tickers con mas de 800 registros
ticker_top=list(df_counts[df_counts.ticker>800].index)
# Lista 'top' tickers en minuscula
ticker_top_low=[]
for word in ticker_top:
  ticker_top_low.append(word.lower())
len(ticker_top)

40

In [156]:
# Lista tickers en minuscula
ticker_list=set(ticker_list)
ticker_low_list=[]
for word in ticker_list:
  ticker_low_list.append(word.lower())
len(ticker_list)

388

In [157]:
# Diccionario con vector de embeddings para los tickers seleccionados
embeddings_tickers= {}
count_no_encn=0
count_encn=0
for tick in ticker_low_list:
  try:
    embeddings_tickers[tick.upper()]=embeddings[tick]
    count_encn+=1
  except:
    count_no_encn+=1  
print(count_no_encn)
print(count_encn)

10
378


In [158]:
# Dataframe con tickers y embeddings
df_ticker= (pd.DataFrame(embeddings_tickers)).T.reset_index().rename(columns={'index':'tickers'})
df_ticker.head()

,tickers,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,SPGI,0.584564,2.216319,1.935004,2.015835,2.368881,1.732645,1.780420,-0.967393,-1.536189,-1.320857,-4.832930,0.718732,-1.224573,-2.717190,1.298834,0.869890,-2.762770,-1.240653,-0.645811,0.993639,-1.231071,-1.405677,2.234833,1.516477,0.872348,-2.004479,1.244900,1.158597,0.289334,0.354488,1.198745,2.125031,1.043576,-0.796886,0.403814,0.994817,-0.949282,-2.968786,-1.378741,-0.574627,-1.912701,3.636627,2.341389,-1.003874,1.629686,-3.092700,-0.730019,-0.534505,1.480113,...,1.830995,-3.118619,0.062808,2.648552,-1.610808,4.293792,1.636358,-0.533970,-1.829359,0.868063,-3.431865,-0.156054,0.771195,1.109036,1.128320,-1.111910,-0.329911,0.085003,-1.163452,-1.074318,-1.312150,1.879428,3.196213,-3.716517,0.666764,-2.053659,-0.645031,1.246101,0.559775,-0.524007,0.991830,-0.619562,-2.440879,-5.236860,-5.136682,1.397233,1.067374,-0.994883,0.502498,-1.046842,-1.771040,-2.902447,-1.832420,-2.125515,1.426445,2.651506,0.517839,0.757128,2.551112,2.312111
1,TSLA,-2.279429,2.454614,-0.762606,2.240674,-0.816147,-0.375941,-0.886259,0.857278,-1.355634,-1.364968,-2.549090,-3.779485,0.240093,3.138210,1.372476,3.989936,0.238906,-2.081098,1.168638,-0.118580,2.381252,3.439993,3.739950,-2.855067,-0.741166,1.378477,0.901401,3.410915,-1.674394,-1.819856,1.124463,-3.328674,-1.807618,-3.831665,-0.162033,0.270124,-3.412543,4.953801,2.653123,-3.910193,-2.848558,-1.741503,3.139614,0.295667,0.847659,2.499944,-0.634010,0.988983,-1.992059,...,0.102597,3.249620,-0.383961,-0.519367,-1.043071,1.642625,1.006046,3.031872,-0.549446,-2.651946,-1.046821,0.864738,2.045406,0.299244,0.531395,-3.080444,-2.767721,-5.715636,-1.228449,0.463025,0.007146,-0.970956,-2.660038,4.442138,-0.873324,1.712502,-0.406247,2.717485,3.146441,1.102844,-2.202104,-2.516624,-0.038036,3.411603,-6.468085,2.748619,-1.591375,3.955458,0.775882,-3.501123,0.282825,-2.145495,2.123845,2.716302,2.679642,-1.774279,-0.410677,1.523367,-0.851353,0.775123
2,SPG,-0.905369,-0.308989,-1.208857,1.822976,0.435876,-1.074843,0.952685,-0.430258,-0.251471,-0.128218,-2.641558,1.212651,-0.183659,-1.754364,0.325166,1.316231,-0.745590,-0.165524,1.302487,-0.736708,0.814732,1.250324,0.060671,0.817414,-0.065847,1.028323,1.166379,2.080573,2.088042,0.133783,0.185683,0.506981,0.124357,0.396960,0.621757,-0.271907,0.496050,0.457276,2.233980,0.192430,0.285843,1.741282,-1.572048,-1.324564,-1.316493,0.009102,0.950724,1.652520,-1.571607,...,0.617790,-1.869228,-0.706727,-2.142147,0.884665,1.811015,1.926556,-0.613805,-3.246145,0.763917,-0.594039,1.060399,0.023558,0.225716,0.237846,-1.392041,-0.151585,-0.349129,0.396069,-0.373570,0.061017,-1.363445,-0.629769,-1.031694,1.077862,-1.542661,-0.753573,-0.434082,2.272252,-0.157143,-0.458252,0.728254,0.027548,-0.994019,-0.951395,1.764622,0.075112,0.845785,0.164905,-0.287887,-0.895544,0.613934,-0.628555,-2.496558,0.453206,-0.411603,0.209080,-1.190446,0.734186,-0.179891
3,XOM,0.406196,0.956807,4.362878,0.584403,0.590328,3.956650,0.525300,-1.685973,-3.690191,-1.152242,-3.267302,-2.122980,-1.278666,-1.093485,-1.758201,-1.992385,-2.164546,-2.273633,-1.359788,-1.497359,-1.231175,-0.776456,1.067042,0.971891,-0.622447,1.750792,-1.001954,3.574520,2.363664,-1.628032,-0.255936,0.883406,-2.590944,-3.325303,-0.244374,2.096876,2.216268,-1.724949,2.234206,-1.141690,-0.101727,3.873194,0.312559,2.478765,-0.599572,-2.062701,-2.280360,-2.022339,3.856195,...,-1.070247,-1.625412,-0.342858,-0.870991,-2.120183,1.010084,1.106946,-1.027689,-1.788003,-2.209626,0.226617,2.032626,1.577073,3.042961,0.846853,-0.921574,0.448396,-2.143201,-1.006290,-0.664726,2.772041,-0.046761,-0.089515,-0.724452,0.151583,-1.334546,1.758008,1.369570,3.163034,-0.823771,-1.201611,-3.098417,-3.038254,-1.697123,-1.572509,-1.472972,0.384895,-1.581620,2.37664

In [159]:
# Dataframe con embeddings
X_ticker=df_ticker.copy().drop(['tickers'],axis=1)
X_ticker.shape

(378, 100)

Escalar valores

In [160]:
from sklearn import preprocessing
import numpy as np

scaler = preprocessing.StandardScaler().fit(X_ticker)
X_scaled = scaler.transform(X_ticker)
X_scaled.shape

(378, 100)

Proyeccion al plano con t-SNE

In [161]:
from sklearn import (manifold, preprocessing, decomposition)
import time

In [162]:
time_start = time.time()

tsne = manifold.TSNE(n_components=2, verbose=1,perplexity=50, n_iter=5000)
X_tsne = tsne.fit_transform(X_scaled)

print ('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 378 samples in 0.004s...
[t-SNE] Computed neighbors for 378 samples in 0.043s...
[t-SNE] Computed conditional probabilities for sample 378 / 378
[t-SNE] Mean sigma: 3.373401
[t-SNE] KL divergence after 250 iterations with early exaggeration: 75.977058
[t-SNE] KL divergence after 1350 iterations: 0.782076
t-SNE done! Time elapsed: 3.5935423374176025 seconds


In [163]:
X_tsne.shape

(378, 2)

In [164]:
from plotly.offline import init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import seaborn as sns
 

Grafico con las proyecciones de tickers utilizando t-SNE

In [165]:
data=[]
bool_crack=df_ticker["tickers"].isin(ticker_top) #bool top tickers

#graficamos los datos en el espacio transformado: las dos componentes tsne

graf1 = go.Scatter(x=X_tsne[:,0], y=X_tsne[:,1],
                           mode='markers',
                        text=df_ticker.loc[:,'tickers'], #'Height', 'Weight', 'Club', 'Age', 'Name','Position'
                           marker=dict(
                                size=5)
                           )

crack =go.Scatter(x=X_tsne[bool_crack,0], y=X_tsne[bool_crack,1],name='Top tickers',
                      text=df_ticker.loc[bool_crack,'tickers'],
                      textfont=dict(family='sans serif',size=10,color='black'),
                      opacity=0.9,mode='text')

data=[graf1,crack]


layout = go.Layout(title="Visualización de la base en el espacio tsne",titlefont=dict(size=20),
                xaxis=dict(title='componente 1'),
                yaxis=dict(title='componente 2'),
                autosize=False, width=1000,height=700)

fig_1 = go.Figure(data=data, layout=layout)

#iplot(fig)
#plot(fig)

In [215]:
iplot(fig_1)
plot(fig_1)

'temp-plot.html'

In [166]:
#En Colab
fig_1.show(renderer="colab")

### Sectores

¿Tienen relacion estos agrupamientos con el sector al cual pertenecen las compañias?

In [167]:
url='/content/drive/MyDrive/Mentoria/dict_tick_sector.json'

In [168]:
import json
with open(url) as file:
    data_sector = json.load(file)

In [169]:
def sector(ticker):
  try:
    return data_sector[ticker.lower()]
  except:
    return 'nan'

In [170]:
df_ticker["Sector"]=df_ticker["tickers"].apply(lambda x: sector(x.lower()))

df_ticker["Sector"].value_counts()

consumer cyclical         59
technology                58
industrials               52
healthcare                47
financial services        43
consumer defensive        33
energy                    22
basic materials           22
nan                       18
communication services     9
utilities                  9
real estate                6
Name: Sector, dtype: int64

In [171]:
palette=['navy','red','yellow','orange','green','brown','blue','magenta','goldenrod','black','gray','pink'] 

In [172]:
bool_technology= df_ticker["Sector"] == 'technology'
bool_financial= df_ticker["Sector"] == 'financial services'
bool_industrials= df_ticker["Sector"] == 'industrials'
bool_healthcare= df_ticker["Sector"] == 'healthcare'
bool_consumer_cyclical= df_ticker["Sector"] == 'consumer cyclical'
bool_consumer_defensive= df_ticker["Sector"] == 'consumer defensive'
bool_energy= df_ticker["Sector"] == 'energy'
bool_basic_materials= df_ticker["Sector"] == 'basic materials'
bool_communication= df_ticker["Sector"] == 'communication services'
bool_utilities= df_ticker["Sector"] == 'utilities'
bool_real= df_ticker["Sector"] == 'real estate'


bool_crack=df_ticker["tickers"].isin(ticker_top)

#palette=['navy','red','#A2D5F2','orange','green','pink']  
data=[]

technology =go.Scatter(x=X_tsne[bool_technology,0], y=X_tsne[bool_technology,1],name='technology',
                      text=df_ticker.loc[bool_technology,'tickers'],
                      opacity=0.9,marker=dict(color=palette[2],size=5),mode='markers')

financial =go.Scatter(x=X_tsne[bool_financial,0], y=X_tsne[bool_financial,1],name='financial',
                      text=df_ticker.loc[bool_financial,'tickers'],
                      opacity=0.6,marker=dict(color=palette[1],size=5),mode='markers')

industrials =go.Scatter(x=X_tsne[bool_industrials,0], y=X_tsne[bool_industrials,1],name='industrials',
                      text=df_ticker.loc[bool_industrials,'tickers'],
                      opacity=0.7,marker=dict(color=palette[3],size=5),mode='markers')

healthcare =go.Scatter(x=X_tsne[bool_healthcare,0], y=X_tsne[bool_healthcare,1],name='healthcare',
                      text=df_ticker.loc[bool_healthcare,'tickers'],
                      opacity=0.4,marker=dict(color=palette[5],size=5),mode='markers')

consumer_cyclical =go.Scatter(x=X_tsne[bool_consumer_cyclical,0], y=X_tsne[bool_consumer_cyclical,1],name='consumer cyclical',
                      text=df_ticker.loc[bool_consumer_cyclical,'tickers'],
                      opacity=0.9,marker=dict(color=palette[6],size=5),mode='markers')

consumer_defensive =go.Scatter(x=X_tsne[bool_consumer_defensive,0], y=X_tsne[bool_consumer_defensive,1],name='consumer defensive',
                      text=df_ticker.loc[bool_consumer_defensive,'tickers'],
                      opacity=0.9,marker=dict(color=palette[7],size=5),mode='markers')

energy =go.Scatter(x=X_tsne[bool_energy,0], y=X_tsne[bool_energy,1],name='energy',
                      text=df_ticker.loc[bool_energy,'tickers'],
                      opacity=0.9,marker=dict(color=palette[8],size=5),mode='markers')

basic_materials =go.Scatter(x=X_tsne[bool_basic_materials,0], y=X_tsne[bool_basic_materials,1],name='basic materials',
                      text=df_ticker.loc[bool_basic_materials,'tickers'],
                      opacity=0.9,marker=dict(color=palette[0],size=5),mode='markers')


communication =go.Scatter(x=X_tsne[bool_communication,0], y=X_tsne[bool_communication,1],name='communication services',
                      text=df_ticker.loc[bool_communication,'tickers'],
                      opacity=0.9,marker=dict(color=palette[9],size=5),mode='markers')


utilities =go.Scatter(x=X_tsne[bool_utilities,0], y=X_tsne[bool_utilities,1],name='utilities',
                      text=df_ticker.loc[bool_utilities,'tickers'],
                      opacity=0.9,marker=dict(color=palette[10],size=5),mode='markers')


real =go.Scatter(x=X_tsne[bool_real,0], y=X_tsne[bool_real,1],name='real estate',
                      text=df_ticker.loc[bool_real,'tickers'],
                      opacity=0.9,marker=dict(color=palette[11],size=5),mode='markers')

crack =go.Scatter(x=X_tsne[bool_crack,0], y=X_tsne[bool_crack,1],name='Top',
                      text=df_ticker.loc[bool_crack,'tickers'],
                      textfont=dict(family='sans serif',size=10,color='black'),
                      opacity=0.9,mode='text')

data=[technology,financial,industrials,healthcare,consumer_cyclical,consumer_defensive,energy,basic_materials,crack]

layout = go.Layout(title="t-SNE - Tickers News",titlefont=dict(size=20),
                autosize=False, width=1150,height=750)

fig_1_sector = go.Figure(data=data, layout=layout)

#iplot(fig)
#plot(fig)

In [214]:
iplot(fig_1_sector)
plot(fig_1_sector)

'temp-plot.html'

In [173]:
#En Colab
fig_1_sector.show(renderer="colab")

## TSNE agregando a cada ticker el nombre de la compañia

Agregar a cada ticker el vector que corresponde al nombre de la compañia,hacer un promedio de estos vectores, para ver si de esta forma se obtiene otra representacion que contenga mas informacion sobre cada ticker.


In [174]:
url= '/content/drive/MyDrive/Mentoria/symbols_tickers.json' #'Datasets/symbols_tickers.json'

In [175]:
# Archivo json con nombre de compañias segun tickers
import json
with open(url) as file:
    data = json.load(file)

In [176]:
def name_ticker(ticker):
    for i in range(len(data)):
        if (data[i]['symbol']==ticker):
            return(data[i]['name'])
       
    return -1
    
name_ticker('EL')

'ESTEE LAUDER COMPANIES-CL A'

In [177]:
# Terminos que podria usar, o no, para limpiar nombres
names_clean=['inc','corp' ,'co' ,'ltd','group' ,'cl' ,'co-cl','plc-class','&' ,'company' ,'plc','pcl' ,'in' ]

In [178]:
names_clean_in_emb=[]
for word in names_clean:
  #print(word)
  try:
    embeddings[word]
    names_clean_in_emb.append(word)
    #print(word, 'esta')
  except:
    #print(word, 'No esta')
    continue

In [179]:
names_clean_in_emb

['inc', 'corp', 'co', 'ltd', 'group', 'cl', 'company', 'plc', 'pcl']

In [180]:
# Crear lista de ticker y nombres de compañias
dict_ticker_company={}
list_no_company=[]
count=0
for ticker in ticker_list:
    #list_ticker_company.append(ticker.lower())
    company_name=name_ticker(ticker)
    if (company_name!=-1):
        company_name_list=company_name
        dict_ticker_company[ticker.lower()]=company_name_list.lower()+' '+ ticker.lower()
        #for word in company_name_list:
        #     dict_ticker_company.append(word.lower())
            #dict_ticker_company.append(word)
            #dict_ticker_company.append(primer_mayuscula(loword))
    else:
        dict_ticker_company[ticker.lower()]=ticker.lower()
        list_no_company.append(ticker)
        count+=1

In [181]:
#dict_ticker_company

In [182]:
len(dict_ticker_company) , len(ticker_list)

(388, 388)

In [199]:
# Crear diccionario con vectores que forman el nombre de compania
emb_tick_with_comp_names= {}
list_no_encn=[]
count_no_encn=0
count_encn=0

for tick in ticker_low_list:
  
    dict_content= list(set(dict_ticker_company[tick].split())-set(set(dict_ticker_company[tick].split())&set(names_clean_in_emb)))
    temp_array=[]
    #print(dict_content)
    for word in dict_content:
      try:
        temp_array.append(embeddings[word])
        #print(temp_array)
        count_encn+=1
      except:
        list_no_encn.append(word)
        count_no_encn+=1
      #print(len(temp_array))
    if (len(temp_array)!=0):
      temp_array_np=np.array(temp_array)      
      emb_tick_with_comp_names[tick.upper()]=np.average(temp_array_np,axis=0) # Suma, promedio ?
    
   
print(count_no_encn)
print(count_encn)

149
965


In [200]:
# ticker no encontrados en embedding
len(set(list_no_encn) & set(ticker_low_list))

10

In [201]:
len(emb_tick_with_comp_names)

383

In [202]:
embeddings.similar_by_vector(emb_tick_with_comp_names['GOLD'])

[('gold', 0.9077646732330322),
 ('silver', 0.8692494630813599),
 ('barrick', 0.8040914535522461),
 ('abx', 0.757229208946228),
 ('mag', 0.7350956797599792),
 ('copper', 0.7165840864181519),
 ('yamana', 0.7070299983024597),
 ('newmont', 0.6987410187721252),
 ('osisko', 0.692679762840271),
 ('asanko', 0.6890450716018677)]

In [203]:
# Df con vectores para tickers
df_ticker_with_name= (pd.DataFrame(emb_tick_with_comp_names)).T.reset_index().rename(columns={'index':'tickers'})
df_ticker_with_name.head()

,tickers,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,SPGI,2.563143,1.365254,1.112893,0.482195,0.607864,2.266586,1.934914,-2.301006,-0.549139,-0.782721,-2.855089,0.580298,-0.671509,-0.297220,1.388119,-1.260170,-1.707758,-0.566863,-1.021444,0.991938,0.083334,-0.493793,1.285335,-0.971600,2.484224,-0.546765,2.034072,0.678233,-0.529299,1.031429,0.622428,0.963141,1.564303,0.072136,0.229229,-0.922197,-1.053265,-1.273172,-3.855770,0.483685,-2.811294,2.984292,1.563438,-0.794764,0.451676,-2.839190,-0.922183,-0.588937,0.124389,...,2.212307,-1.373652,-1.151628,1.233082,-0.114739,0.355403,-0.222574,-0.024164,0.186288,0.312657,-2.103112,-0.771567,-0.795334,0.670436,0.906847,-0.556761,0.320276,-0.397227,-0.685960,0.072995,-0.722636,1.684687,1.527135,-2.895229,-0.952046,-1.625690,-0.077400,0.343466,-0.827054,-0.101974,-0.986857,0.275838,-0.638694,-3.272548,-2.409031,0.718897,-0.527748,-0.067337,-1.780519,-0.882830,-1.632579,-2.657023,-2.129220,-0.115562,1.324161,0.519318,-1.152068,-0.380904,3.038339,0.526232
1,TSLA,-2.006270,1.535045,-1.276422,2.253911,-1.281605,-0.170470,-0.618417,0.629589,-1.184561,-0.704891,-1.520849,-1.792868,-0.692392,2.243218,0.866788,2.172557,1.277642,-1.638835,0.348648,2.151544,1.861925,2.886126,2.660352,-1.536158,-0.842763,1.544363,0.778875,3.608381,-1.977650,-1.690201,1.120518,-3.466712,-1.741359,-3.111041,-1.273530,0.209606,-2.188210,3.417231,2.698741,-3.866396,-1.357196,-1.541459,2.314307,0.428093,0.162664,1.642685,-0.839555,0.334394,-0.236176,...,-0.861562,3.052371,-1.081948,-0.050591,-0.999002,1.286380,1.511587,2.777957,-0.820142,-1.556509,0.206523,0.437074,1.488777,0.608497,0.024010,-3.042209,-3.219231,-4.476652,-1.376558,-0.748353,0.574541,-1.057215,-1.328849,2.984276,-1.021179,1.553032,-0.144159,3.502051,1.707364,0.913523,-1.849349,-0.877244,-0.427580,2.878382,-4.701870,2.158471,-1.405676,3.185790,0.158512,-2.890890,0.355220,-1.208598,2.342035,2.049306,2.368890,-0.401143,-0.147195,0.874461,-0.836338,1.511108
2,SPG,-0.186339,0.432199,-2.382167,1.464005,-0.498271,-2.232544,0.705194,-0.440873,0.414185,0.229355,-1.657238,1.956500,-0.936880,-0.249881,1.009666,1.545670,0.356428,1.262355,1.255779,0.118968,1.350170,1.588106,-0.774770,-0.184118,0.739551,-0.226639,1.564503,2.049575,2.719796,1.337711,-0.409861,0.258635,0.423452,2.122319,-0.189819,-0.666852,1.398204,1.616932,2.961853,0.214020,-0.612610,1.164698,-1.542820,-1.773820,-1.163510,0.494149,0.786239,2.534072,-1.081004,...,0.228059,-0.222501,-1.273461,-2.174260,1.661662,1.489016,1.663661,-0.615389,-3.350063,1.573433,-1.215588,-0.283187,-0.595150,-0.775193,0.073516,-0.055402,-0.330044,-0.043083,-0.068008,-0.167921,0.012010,-2.808956,-0.979884,-2.480236,0.220134,-1.080997,-0.693541,0.094829,2.029643,0.152247,-1.094119,0.584320,0.135645,-0.690106,0.049924,0.892750,-1.240909,0.469891,-0.016474,0.788988,0.470190,0.800081,-0.855973,-2.182016,-0.092814,-0.339801,-1.131242,-2.170830,1.973664,0.079429
3,XOM,0.549840,1.472050,3.649642,0.517277,1.131789,4.100861,-0.433317,-3.080845,-2.163394,-1.008068,-2.526573,-2.262453,-1.840403,-0.610588,-1.403553,-1.775790,-1.862253,-1.899588,-1.620691,-1.274856,-1.258902,-0.468510,0.746549,0.366422,-0.667172,1.052715,-0.972872,3.536357,1.048322,-1.568869,0.217750,1.065026,-1.604079,-2.695091,0.480879,1.741626,2.788656,-1.173283,1.820558,-1.176862,0.303164,1.628694,0.413759,2.874446,0.008407,-1.722086,-1.681245,-1.788172,4.214243,...,-1.074535,-0.498597,0.174571,-1.332097,-1.967473,0.808376,0.397472,-0.770224,-0.247794,-2.771660,0.372336,1.294655,0.889683,3.081977,0.633862,-0.286723,0.883439,-2.192332,-0.745756,-1.289132,2.709081,0.024834,-0.871629,-0.706097,-0.066508,-1.204489,1.841466,1.079788,3.248893,-0.424479,-2.591975,-3.782728,-2.426746,-0.563153,-1.211615,-1.394256,0.269004,-1.367368,

In [204]:
df_ticker_with_name.isna().value_counts()

tickers  0      1      2      3      4      5      6      7      8      9      10     11     12     13     14     15     16     17     18     19     20     21     22     23     24     25     26     27     28     29     30     31     32     33     34     35     36     37     38     39     40     41     42     43     44     45     46     47     48     49     50     51     52     53     54     55     56     57     58     59     60     61     62     63     64     65     66     67     68     69     70     71     72     73     74     75     76     77     78     79     80     81     82     83     84     85     86     87     88     89     90     91     92     93     94     95     96     97     98     99   
False    False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  Fal

In [205]:
# Matriz de embeddings
X_ticker_with_name=df_ticker_with_name.copy().drop(['tickers'],axis=1)
X_ticker_with_name.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,2.563143,1.365254,1.112893,0.482195,0.607864,2.266586,1.934914,-2.301006,-0.549139,-0.782721,-2.855089,0.580298,-0.671509,-0.297220,1.388119,-1.260170,-1.707758,-0.566863,-1.021444,0.991938,0.083334,-0.493793,1.285335,-0.971600,2.484224,-0.546765,2.034072,0.678233,-0.529299,1.031429,0.622428,0.963141,1.564303,0.072136,0.229229,-0.922197,-1.053265,-1.273172,-3.855770,0.483685,-2.811294,2.984292,1.563438,-0.794764,0.451676,-2.839190,-0.922183,-0.588937,0.124389,0.587158,2.212307,-1.373652,-1.151628,1.233082,-0.114739,0.355403,-0.222574,-0.024164,0.186288,0.312657,-2.103112,-0.771567,-0.795334,0.670436,0.906847,-0.556761,0.320276,-0.397227,-0.685960,0.072995,-0.722636,1.684687,1.527135,-2.895229,-0.952046,-1.625690,-0.077400,0.343466,-0.827054,-0.101974,-0.986857,0.275838,-0.638694,-3.272548,-2.409031,0.718897,-0.527748,-0.067337,-1.780519,-0.882830,-1.632579,-2.657023,-2.129220,-0.115562,1.324161,0.519318,-1.152068,-0.380904,3.038339,0.526232
1,-2.006270,1.535045,-1.276422,2.253911,-1.281605,-0.170470,-0.618417,0.629589,-1.184561,-0.704891,-1.520849,-1.792868,-0.692392,2.243218,0.866788,2.172557,1.277642,-1.638835,0.348648,2.151544,1.861925,2.886126,2.660352,-1.536158,-0.842763,1.544363,0.778875,3.608381,-1.977650,-1.690201,1.120518,-3.466712,-1.741359,-3.111041,-1.273530,0.209606,-2.188210,3.417231,2.698741,-3.866396,-1.357196,-1.541459,2.314307,0.428093,0.162664,1.642685,-0.839555,0.334394,-0.236176,-0.483816,-0.861562,3.052371,-1.081948,-0.050591,-0.999002,1.286380,1.511587,2.777957,-0.820142,-1.556509,0.206523,0.437074,1.488777,0.608497,0.024010,-3.042209,-3.219231,-4.476652,-1.376558,-0.748353,0.574541,-1.057215,-1.328849,2.984276,-1.021179,1.553032,-0.144159,3.502051,1.707364,0.913523,-1.849349,-0.877244,-0.427580,2.878382,-4.701870,2.158471,-1.405676,3.185790,0.158512,-2.890890,0.355220,-1.208598,2.342035,2.049306,2.368890,-0.401143,-0.147195,0.874461,-0.836338,1.511108
2,-0.186339,0.432199,-2.382167,1.464005,-0.498271,-2.232544,0.705194,-0.440873,0.414185,0.229355,-1.657238,1.956500,-0.936880,-0.249881,1.009666,1.545670,0.356428,1.262355,1.255779,0.118968,1.350170,1.588106,-0.774770,-0.184118,0.739551,-0.226639,1.564503,2.049575,2.719796,1.337711,-0.409861,0.258635,0.423452,2.122319,-0.189819,-0.666852,1.398204,1.616932,2.961853,0.214020,-0.612610,1.164698,-1.542820,-1.773820,-1.163510,0.494149,0.786239,2.534072,-1.081004,0.704697,0.228059,-0.222501,-1.273461,-2.174260,1.661662,1.489016,1.663661,-0.615389,-3.350063,1.573433,-1.215588,-0.283187,-0.595150,-0.775193,0.073516,-0.055402,-0.330044,-0.043083,-0.068008,-0.167921,0.012010,-2.808956,-0.979884,-2.480236,0.220134,-1.080997,-0.693541,0.094829,2.029643,0.152247,-1.094119,0.584320,0.135645,-0.690106,0.049924,0.892750,-1.240909,0.469891,-0.016474,0.788988,0.470190,0.800081,-0.855973,-2.182016,-0.092814,-0.339801,-1.131242,-2.170830,1.973664,0.079429
3,0.549840,1.472050,3.649642,0.517277,1.131789,4.100861,-0.433317,-3.080845,-2.163394,-1.008068,-2.526573,-2.262453,-1.840403,-0.610588,-1.403553,-1.775790,-1.862253,-1.899588,-1.620691,-1.274856,-1.258902,-0.468510,0.746549,0.366422,-0.667172,1.052715,-0.972872,3.536357,1.048322,-1.568869,0.217750,1.065026,-1.604079,-2.695091,0.480879,1.741626,2.788656,-1.173283,1.820558,-1.176862,0.303164,1.628694,0.413759,2.874446,0.008407,-1.722086,-1.681245,-1.788172,4.214243,0.195302,-1.074535,-0.498597,0.174571,-1.332097,-1.967473,0.808376,0.397472,-0.770224,-0.247794,-2.771660,0.372336,1.294655,0.889683,3.081977,0.633862,-0.286723,0.883439,-2.192332,-0.745756,-1.289132,2.709081,0.024834,-0.871629,-0.706097,-0.066508,-1.204489,1.841466,1.079788,3.248893,-0.424479,-2.591975,-3.782728,-2.426746,-0.563153,-1.211615,-1.394256,0.269004,-1.367368,1.2572

In [206]:
# Escalado
scaler_2 = preprocessing.StandardScaler().fit(X_ticker_with_name)
X_scaled_name = scaler_2.transform(X_ticker_with_name)
X_scaled_name.shape

(383, 100)

In [207]:
time_start = time.time()

tsne = manifold.TSNE(n_components=2, verbose=1,perplexity=30, n_iter=5000)
X_tsne_with_names = tsne.fit_transform(X_ticker_with_name)

print ('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 383 samples in 0.003s...
[t-SNE] Computed neighbors for 383 samples in 0.035s...
[t-SNE] Computed conditional probabilities for sample 383 / 383
[t-SNE] Mean sigma: 4.148042
[t-SNE] KL divergence after 250 iterations with early exaggeration: 89.996201
[t-SNE] KL divergence after 2550 iterations: 1.067645
t-SNE done! Time elapsed: 5.542469263076782 seconds


In [208]:
data=[]
bool_crack=df_ticker_with_name["tickers"].isin(ticker_top) #top tickers

#graficamos los datos en el espacio transformado: las dos componentes tsne

graf1 = go.Scatter(x=X_tsne_with_names[:,0], y=X_tsne_with_names[:,1],name='Tickers',
                           mode='markers',
                        text=df_ticker_with_name.loc[:,'tickers'], #'Height', 'Weight', 'Club', 'Age', 'Name','Position'
                           marker=dict(
                                size=5)
                           )

crack =go.Scatter(x=X_tsne_with_names[bool_crack,0], y=X_tsne_with_names[bool_crack,1],name='Top Tickers',
                      text=df_ticker_with_name.loc[bool_crack,'tickers'],
                      textfont=dict(family='sans serif',size=12,color='black'),
                      opacity=0.9,mode='text')

data=[graf1,crack]


layout = go.Layout(title="Visualización de la base en el espacio tsne",titlefont=dict(size=20),
                xaxis=dict(title='componente 1'),
                yaxis=dict(title='componente 2'),
                autosize=False, width=1000,height=700)

fig_2 = go.Figure(data=data, layout=layout)

#iplot(fig)
#plot(fig)

In [216]:

iplot(fig_2)
plot(fig_2)

'temp-plot.html'

In [209]:
#En Colab
fig_2.show(renderer="colab")

### Sectores

Agregamos sectores al grafico anterior

In [210]:
df_ticker_with_name["Sector"]=df_ticker_with_name["tickers"].apply(lambda x: sector(x))

df_ticker_with_name["Sector"].value_counts()

consumer cyclical         60
technology                58
industrials               52
healthcare                48
financial services        45
consumer defensive        33
energy                    22
basic materials           22
nan                       18
utilities                 10
communication services     9
real estate                6
Name: Sector, dtype: int64

In [211]:
bool_technology= df_ticker_with_name["Sector"] == 'technology'
bool_financial= df_ticker_with_name["Sector"] == 'financial services'
bool_industrials= df_ticker_with_name["Sector"] == 'industrials'

bool_healthcare= df_ticker_with_name["Sector"] == 'healthcare'
bool_consumer_cyclical= df_ticker_with_name["Sector"] == 'consumer cyclical'
bool_consumer_defensive= df_ticker_with_name["Sector"] == 'consumer defensive'
bool_energy= df_ticker_with_name["Sector"] == 'energy'
bool_basic_materials= df_ticker_with_name["Sector"] == 'basic materials'

bool_crack=df_ticker_with_name["tickers"].isin(ticker_top)

#palette=['navy','red','#A2D5F2','orange','green','pink']  
data=[]

technology =go.Scatter(x=X_tsne_with_names[bool_technology,0], y=X_tsne_with_names[bool_technology,1],name='technology',
                      text=df_ticker_with_name.loc[bool_technology,'tickers'],
                      opacity=0.9,marker=dict(color=palette[2],size=5),mode='markers')

financial =go.Scatter(x=X_tsne_with_names[bool_financial,0], y=X_tsne_with_names[bool_financial,1],name='financial',
                      text=df_ticker_with_name.loc[bool_financial,'tickers'],
                      opacity=0.6,marker=dict(color=palette[1],size=5),mode='markers')

industrials =go.Scatter(x=X_tsne_with_names[bool_industrials,0], y=X_tsne_with_names[bool_industrials,1],name='industrials',
                      text=df_ticker_with_name.loc[bool_industrials,'tickers'],
                      opacity=0.7,marker=dict(color=palette[3],size=5),mode='markers')

healthcare =go.Scatter(x=X_tsne_with_names[bool_healthcare,0], y=X_tsne_with_names[bool_healthcare,1],name='healthcare',
                      text=df_ticker_with_name.loc[bool_healthcare,'tickers'],
                      opacity=0.4,marker=dict(color=palette[5],size=5),mode='markers')

consumer_cyclical =go.Scatter(x=X_tsne_with_names[bool_consumer_cyclical,0], y=X_tsne_with_names[bool_consumer_cyclical,1],name='consumer cyclical',
                      text=df_ticker_with_name.loc[bool_consumer_cyclical,'tickers'],
                      opacity=0.9,marker=dict(color=palette[6],size=5),mode='markers')

consumer_defensive =go.Scatter(x=X_tsne_with_names[bool_consumer_defensive,0], y=X_tsne_with_names[bool_consumer_defensive,1],name='consumer defensive',
                      text=df_ticker_with_name.loc[bool_consumer_defensive,'tickers'],
                      opacity=0.9,marker=dict(color=palette[7],size=5),mode='markers')

energy =go.Scatter(x=X_tsne_with_names[bool_energy,0], y=X_tsne_with_names[bool_energy,1],name='energy',
                      text=df_ticker_with_name.loc[bool_energy,'tickers'],
                      opacity=0.9,marker=dict(color=palette[8],size=5),mode='markers')

basic_materials =go.Scatter(x=X_tsne_with_names[bool_basic_materials,0], y=X_tsne_with_names[bool_basic_materials,1],name='basic materials',
                      text=df_ticker_with_name.loc[bool_basic_materials,'tickers'],
                      opacity=0.9,marker=dict(color=palette[0],size=5),mode='markers')

crack =go.Scatter(x=X_tsne_with_names[bool_crack,0], y=X_tsne_with_names[bool_crack,1],name='Top',
                      text=df_ticker_with_name.loc[bool_crack,'tickers'],
                      textfont=dict(family='sans serif',size=12,color='black'),
                      opacity=0.9,mode='text')

data=[technology,financial,industrials,healthcare,energy,basic_materials,crack,consumer_cyclical,consumer_defensive] # 

layout = go.Layout(title="t-SNE - News Tickers",titlefont=dict(size=20),
                autosize=False, width=1150,height=750)

fig_3_sector = go.Figure(data=data, layout=layout)

In [217]:

iplot(fig_3_sector)
plot(fig_3_sector)

'temp-plot.html'

In [212]:
#En Colab
fig_3_sector.show(renderer="colab")

En el grafico anterior podemos observar 'que aprende' , la representacion en vectores realizada con word2vec, sobre las compañias que son mencionados en las noticias financieras. Se observa que las relaciones aprendidas parecen tener alguna relacion con los sectores a los que pertenecen. Empresas tecnologicas como NFLX,AMZN, GOGL, aparecen en el plano mas cerca entre si; al igual que empresas relacionadas al sector financiero como BAC,JPM,WFC.

In [197]:
#import plotly.express as px


#fig_3_sector.write_html("tsne_News_tickers_with_sector.html")

In [198]:
#fig_2.write_html("tsne_News_tickers.html")